# National Park Services (NPS) API Exploration

# README

#### **Goal** 

The goal of this project was to utilize the National Park Services' (NPS) API for purposes of:
   - Practicing data pipe-lining
   - Locally creating datasets derived from API calling
   - Inserting Pandas DataFrames in cloud-based databases (with original intent of AWS specifically)
   - Updating AWS database from Python via use of functions
  
The code contained in this notebook mainly concerns migrating local datasets to a cloud database. Further transformation/normalization/migration occurred mainly in second example below. Due to costs of resources for AWS Redshift, with no trial period available, the first example was only utilized to the point of migrating relational tables to a AWS S3 bucket.  
   - E.g., migrating from local table(s) -> AWS RDS -> AWS S3 -> AWS Redshift -> dbt
   - E.g., migrating from local table(s) -> Snowflake -> dbt

### Commands for local install of pyscopg library & update of pip - without going to Linux terminal

In [ ]:
# importing packages
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

'''
- The following will import an API Key from a function found in a private kaggle notebook... 
  - This was added as a utility script to this notebook & left for demonstration purposes
  - Script is no longer associated with this import, for API security purposes, & the import should be uncommented/replaced if desired to be used
'''
# from npsapikey import get_nps_key

In [ ]:
'''
Imports for:
- External PostgreSQL connections (AWS)
- Snowflake Connector 
'''
import psycopg as ps

# Snowflake module needed for creating connection to Snowflake account & warehouse details
import snowflake.connector
from snowflake.connector.errors import OperationalError
# Snowflake module needed to write Pandas DF to a table
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
'''
The following code should be uncommented & used if utilizing a local Jupyter Notebook...
  - Kaggle utilizes 'Utility Scripts' for ability to import functions from another notebook
  - Local use of Jupyter Notebook utilizes the 'run' magic command to import functions from another notebook
'''

# # importing Jupyter notebook storing API Key
# %run NPSAPIKey.ipynb

# # importing API Key
# API_KEY = get_nps_key()

In [ ]:
# storing API Key from import of Kaggle utility script 
API_KEY = get_nps_key()

In [ ]:
'''
Function for creating 'parks' Pandas DataFrame and storing relative Park details, including: Name, ID, Code, Website URL,
Latitude & Longitude coordinates, physical location's state abbreviation (e.g., NY, NJ)
'''
def get_park_details(parks_df):
    # creating empty list for storing park data
    temp = []
    # making API call
    response = requests.get('https://developer.nps.gov/api/v1/parks?&limit=480&start=0&api_key='+API_KEY).json()
    
    # setting pause-timer to allow enough time for data to flow from API
    time.sleep(2)
    
    for park in response['data']:    
        park_id = park['id']
        park_name = park['fullName']
        park_code = park['parkCode']
        park_url = park['url']
        park_lat = park['latitude']
        park_long = park['longitude']
        park_state = park['addresses'][0]['stateCode']

        # appending park data to empty list
        temp.append({'park_id':park_id, 'park_name': park_name, 'park_code':park_code,
                                    'park_state':park_state, 'park_url':park_url, 'park_lat':park_lat,
                                    'park_long':park_long})

    # creating temporary dataframe from temp list
    temp_df = pd.DataFrame(temp)
    # appending details from temp DF to parks DF 
    parks_df = pd.concat([parks_df, temp_df]).reset_index(drop=True)

    return parks_df

In [ ]:
'''
Function for creating and populating DataFrames for both:
  (1) Unique 'activities' found in 'parks' data received from API call - storing relative Activity's Name and ID
  (2) Parks-Activity relationship - storing associated parkID with activityID for activities available in the associated park
'''
def get_activities_details(activities_df, park_activities_df):
    # empty list for storing activities
    activities_list = []
    # empty list for storing park-activity relationships
    park_act_list = []
    
    # making API call
    resp = requests.get('https://developer.nps.gov/api/v1/parks?&limit=480&start=0&api_key='+API_KEY).json()
    
    # setting pause-timer to allow enough time for data to flow from API
    time.sleep(2)
    
    # loops through parks data and stores list of activities found in each park
    for park in resp['data']:
        # empty list for storing activity IDs
        actid_list = []
        park_id = park['id']
        for activity in park['activities']:
            act_name = activity['name']
            act_id = activity['id']
            # adds activity & asssociated park ID to list if activity ID isn't found in list of activity IDs
            if act_id not in actid_list:
                activities_list.append({'act_id':act_id, 'act_name':act_name})
                park_act_list.append({'park_id': park_id, 'act_id':act_id})
                actid_list.append(act_id)
                
    # creating temporary activity dataframe from temp list
    temp_activity_df = pd.DataFrame(activities_list)
    # creating temporary park-activity dataframe from park-activity relationship list
    temp_park_activity_df = pd.DataFrame(park_act_list)
    
    # appending details from temp activity DF to activities DF 
    activities_df = pd.concat([activities_df, temp_activity_df]).reset_index(drop=True)
    # appending details from temp park-activity relationship DF to park-activity DF 
    park_activities_df = pd.concat([park_activities_df, temp_park_activity_df]).reset_index(drop=True)
    
    return activities_df, park_activities_df

In [ ]:
'''
Function for creating and populating DataFrames for both:
  (1) Unique 'topics' found in 'parks' data received from API call - storing relative Topic's Name and ID
  (2) Parks-Topics relationship, storing associated parkID with topicID for topics available in the associated park
         (e.g., American Revolution, Music, Women's History, etc.) 
'''
def get_topics_details(topics_df, park_topics_df):
    # empty list for storing topics
    topics_list = []
    # empty list for storing park-topics relationships
    park_top_list = []
    
    # making API call
    resp = requests.get('https://developer.nps.gov/api/v1/parks?&limit=480&start=0&api_key='+API_KEY).json()
    
    # setting pause-timer to allow enough time for data to flow from API
    time.sleep(2)
    
    # loops through parks data and stores list of topics found in each park
    for park in resp['data']:
        # empty list for storing topics' IDs
        topid_list = []
        park_id = park['id']
        for topic in park['topics']:
            top_name = topic['name']
            top_id = topic['id']
            # adds topic & asssociated park ID to list if topic ID isn't found in list of topic IDs
            if top_id not in topid_list:
                topics_list.append({'top_id':top_id, 'top_name':top_name})
                park_top_list.append({'park_id': park_id, 'top_id':top_id})
                topid_list.append(top_id)
    
    # creating temporary topics dataframe from temp list
    temp_topics_df = pd.DataFrame(topics_list)
    # creating temporary park-topics dataframe from park-topics relationship list
    temp_park_topics_df = pd.DataFrame(park_top_list)
    
    # appending details from temp topics DF to topics DF 
    topics_df = pd.concat([topics_df, temp_topics_df]).reset_index(drop=True)
    # appending details from temp park-topics relationship DF to park-topics DF 
    park_topics_df = pd.concat([park_topics_df, temp_park_topics_df]).reset_index(drop=True)

    return topics_df, park_topics_df

#### Main

In [ ]:
# building dataframes
parks_df = pd.DataFrame(columns=['park_id', 'park_name', 'park_code', 'park_state', 'park_url', 'park_lat', 'park_long'])
activities_df = pd.DataFrame(columns=['act_id', 'act_name'])
park_activities_df = pd.DataFrame(columns=['park_id', 'act_id'])
topics_df = pd.DataFrame(columns=['top_id', 'top_name'])
park_topics_df = pd.DataFrame(columns=['park_id', 'top_id'])

# calling helper functions to populate created dataframes
parks_df = get_park_details(parks_df)
activities_df, park_activities_df = get_activities_details(activities_df, park_activities_df)
topics_df, park_topics_df = get_topics_details(topics_df, park_topics_df)

In [ ]:
# dropping duplicates from activities DataFrame to ensure only unique values
activities_df = activities_df.drop_duplicates(subset=['act_id']).reset_index(drop=True)

# dropping duplicates from topics DataFrame to ensure only unique values
topics_df = topics_df.drop_duplicates(subset=['top_id']).reset_index(drop=True)

# AWS PostgreSQL Database

## Pushing Pandas Dataframes to AWS PostgreSQL Database

In [ ]:
'''
The following code should be uncommented & used if utilizing a local Jupyter Notebook
  - This is intended to pull stored details for created AWS PostgreSQL database, for remote local connection 
'''

# # importing Jupyter notebook storing NPS DB details
# %run NPSDB.ipynb

# # importing needed DB details
# host, dbname, user, pw, port = get_nps_db()

# #initializing connection variable
# conn=None

In [ ]:
# connects to remote AWS DB, otherwise raises error
try:
    conn = ps.connect(host=host, dbname=dbname, user=user, password=pw, port=port, autocommit=True)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

In [ ]:
# opens cursor to perform database operations
cur = conn.cursor()

### Creating SQL Tables

In [ ]:
# create parks table
create_parks_table = ("""CREATE TABLE IF NOT EXISTS parks (
                    park_id VARCHAR(255) PRIMARY KEY,
                    park_name TEXT NOT NULL,
                    park_state TEXT,
                    park_code VARCHAR(255),
                    park_url TEXT,
                    park_lat VARCHAR(255),
                    park_long VARCHAR(255)
                )""")

# execute sql command
cur.execute(create_parks_table)

In [ ]:
# create activities table if it doesn't exist already
create_activities_table = ("""CREATE TABLE IF NOT EXISTS activities (
                    activity_id VARCHAR(255) PRIMARY KEY,
                    activity_name TEXT NOT NULL
                )""")

# execute sql command
cur.execute(create_activities_table)

In [ ]:
# create parks_activities relationships table
create_park_activities_table = ("""CREATE TABLE IF NOT EXISTS park_activities (
                    park_id VARCHAR(255),
                    activity_id VARCHAR(255),
                    PRIMARY KEY(park_id, activity_id)
                )""")

#execute sql command
cur.execute(create_park_activities_table)

In [ ]:
# create topics table if it doesn't exist already
create_topics_table = ("""CREATE TABLE IF NOT EXISTS topics (
                    topic_id VARCHAR(255) PRIMARY KEY,
                    topic_name TEXT NOT NULL
                )""")

# execute sql command
cur.execute(create_topics_table)

In [ ]:
# create parks_topics relationships table
create_park_topics_table = ("""CREATE TABLE IF NOT EXISTS park_topics (
                    park_id VARCHAR(255),
                    topic_id VARCHAR(255),
                    PRIMARY KEY(park_id, topic_id)
                )""")

#execute sql command
cur.execute(create_park_topics_table)

### Insert & Update Parks Data

In [ ]:
# function for inserting and updating data into parks_df - to be used with 'parks' SQL table
def update_parks_db(cur, parks_df):
    # empty list for storing new IDs
    new_row = []
    
    for i, row in parks_df.iterrows():
        if check_park_existence(cur, row['park_id']): # if park already exists, update SQL table
            update_park_row(cur, row['park_name'], row['park_state'], row['park_code'], row['park_url'], row['park_lat'], row['park_long'], row['park_id'])
        else: # park doesn't exist - append to SQL table
            # append details of new row to list for creating dataframe
            new_row.append(row)
            
    tmp_parks_df = pd.DataFrame(new_row)
    return tmp_parks_df

In [ ]:
# checks if park exists in 'parks' SQL table
def check_park_existence(cur, park_id):
    query = ("""SELECT * FROM parks WHERE park_id = (%s);""")
    cur.execute(query, (park_id,))

    return cur.fetchone() is not None

In [ ]:
# updates park's row in 'parks' SQL table, if already exists
def update_park_row(cur, park_name, park_state, park_code, park_url, park_lat, park_long, park_id):
    query = ("""UPDATE parks SET
                park_name = (%s),
                park_state = (%s),
                park_code = (%s),
                park_url = (%s),
                park_lat = (%s),
                park_long = (%s)
                WHERE park_id = (%s);""")
    params = (park_name, park_state, park_code, park_url, park_lat, park_long, park_id,)

    cur.execute(query, params)

In [ ]:
'''
Function for inserting, row-by-row, the new details contained within new_parks_df
    - This will call the function 'insert_into_parks_table' for actually querying the 'parks' SQL table
'''
def append_parksDF_to_table(cur, new_parks_df):
    # inserts parks row-by-row
    for i, row in new_parks_df.iterrows():
        insert_into_parks_table(cur, row['park_id'], row['park_name'], row['park_state'], row['park_code'], row['park_url'], row['park_lat'], row['park_long'])

In [ ]:
# queries the 'parks' SQL table to insert new row(s) of parks gathered
def insert_into_parks_table(cur, park_id, park_name, park_state, park_code, park_url, park_lat, park_long):
    query = ("""INSERT INTO parks (park_id, park_name, park_state, park_code, park_url, park_lat, park_long)
                VALUES (%s, %s, %s, %s, %s, %s, %s);""")
    params = (park_id, park_name, park_state, park_code, park_url, park_lat, park_long,)

    cur.execute(query, params)

In [ ]:
%%time

# calls function to store data for any new parks
new_parks_df = update_parks_db(cur, parks_df)

# calls function to insert any new rows into 'parks' SQL table
append_parksDF_to_table(cur, new_parks_df)

### Insert & Update Activites Data

In [ ]:
# function for inserting and updating data into activities_df - to be used with 'activities' SQL table
def update_activities_db(cur, activities_df):
    # empty list for storing new IDs
    new_row = []
    
    for i, row in activities_df.iterrows():
        if check_activity_existence(cur, row['act_id']): # if activity already exists, update SQL table
            update_activity_row(cur, row['act_name'])
        else: # activity doesn't exist - append to SQL table
            # append details of new row to list for creating dataframe
            new_row.append(row)
            
    tmp_activity_df = pd.DataFrame(new_row)
    return tmp_activity_df

In [ ]:
# checks if activity exists in 'activities' SQL table
def check_activity_existence(cur, act_id):
    query = ("""SELECT * FROM activities WHERE activity_id = (%s);""")
    cur.execute(query, (act_id,))

    return cur.fetchone() is not None

In [ ]:
# updates activity's row in 'activities' SQL table, if already exists
def update_activity_row(cur, act_name, act_id):
    query = ("""UPDATE activities SET
                activity_name = (%s),
                WHERE activity_id = (%s);""")
    params = (act_name, act_id)

    cur.execute(query, params)

In [ ]:
'''
Function for inserting, row-by-row the new details contained within new_activities_df
    - This will call the function 'insert_into_activities_table' for actually querying the 'activities' SQL table
'''
def append_activitiesDF_to_table(cur, new_activities_df):
    # inserts activities row-by-row
    for i, row in new_activities_df.iterrows():
        insert_into_activities_table(cur, row['act_id'], row['act_name'])

In [ ]:
# queries the 'activities' SQL table to insert new row(s) of activities gathered
def insert_into_activities_table(cur, act_id, act_name):
    query = ("""INSERT INTO activities (activity_id, activity_name)
                VALUES (%s, %s);""")
    params = (act_id, act_name,)

    cur.execute(query, params)

In [ ]:
%%time

# calls function to store data for any new activites
new_activities_df = update_activities_db(cur, activities_df)

# calls function to insert any new rows into 'activities' SQL table
append_activitiesDF_to_table(cur, new_activities_df)

### Insert & Update Topics Data

In [ ]:
# function for inserting and updating data into topics_df - to be used with 'topics' SQL table
def update_topics_db(cur, topics_df):
    # empty list for storing new IDs
    new_row = []
    
    for i, row in topics_df.iterrows():
        if check_topic_existence(cur, row['top_id']): # if topic already exists, update SQL table
            update_topic_row(cur, row['top_name'])
        else: # topic doesn't exist - append to SQL table
            # append details of new row to list for creating dataframe
            new_row.append(row)
            
    tmp_topic_df = pd.DataFrame(new_row)
    return tmp_topic_df

In [ ]:
# checks if topic exists in 'topics' SQL table
def check_topic_existence(cur, top_id):
    query = ("""SELECT * FROM topics WHERE topic_id = (%s);""")
    cur.execute(query, (top_id,))

    return cur.fetchone() is not None

In [ ]:
# updates topic's row in 'topics' SQL table, if already exists
def update_topic_row(cur, top_name, top_id):
    query = ("""UPDATE topics SET
                topic_name = (%s),
                WHERE topic_id = (%s);""")
    params = (top_name, top_id)

    cur.execute(query, params)

In [ ]:
'''
Function for inserting, row-by-row the new details contained within new_topics_df
    - This will call the function 'insert_into_topics_table' for actually querying the 'topics' SQL table
'''
def append_topicDF_to_table(cur, new_topics_df):
    # inserts topics row-by-row
    for i, row in new_topics_df.iterrows():
        insert_into_topics_table(cur, row['top_id'], row['top_name'])

In [ ]:
# queries the 'topics' SQL table to insert new row(s) of topics gathered
def insert_into_topics_table(cur, top_id, top_name):
    query = ("""INSERT INTO topics (topic_id, topic_name)
                VALUES (%s, %s);""")
    params = (top_id, top_name,)

    cur.execute(query, params)

In [ ]:
%%time

# calls function to store data for any new topics
new_topics_df = update_topics_db(cur, topics_df)

# calls function to insert any new rows into 'topics' SQL table
append_topicDF_to_table(cur, new_topics_df)

### Insert & Update Park_Activties Data

In [ ]:
# function for inserting and updating data into park_activities_df - to be used with 'parks_activities' SQL table
def update_park_activities_db(cur, park_activities_df):
    # empty list for storing new IDs
    new_row = []
    
    for i, row in park_activities_df.iterrows():
        if check_park_activity_existence(cur, row['park_id'], row['act_id']) is None: # if park_activity relation doesn't exist - append to SQL table
            # append details of new row to list for creating dataframe
            new_row.append(row)
        else:
            print('Nothing to insert')
            
    tmp_park_activities_df = pd.DataFrame(new_row)
    return tmp_park_activities_df

In [ ]:
# checks if relationship exists in 'park_activities' SQL table
def check_park_activity_existence(cur, park_id, act_id):
    query = ("""SELECT * FROM park_activities WHERE park_id = (%s) AND activity_id = (%s);""")
    cur.execute(query, (park_id, act_id,))

    return cur.fetchone()

In [ ]:
'''
Function for inserting, row-by-row the new details contained within new_parkactivities_df
    - This will call the function 'insert_into_parkactivities_table' for actually querying the 'park_activities' SQL table
'''
def append_parkactivitiesDF_to_table(cur, new_parkactivities_df):
    # inserts park_activity relation row-by-row
    for i, row in new_parkactivities_df.iterrows():
        insert_into_parkactivities_table(cur, row['park_id'], row['act_id'])

In [ ]:
# queries the 'park_activities' SQL table to insert new row(s) of relations gathered
def insert_into_parkactivities_table(cur, park_id, act_id):
    query = ("""INSERT INTO park_activities (park_id, activity_id)
                VALUES (%s, %s);""")
    params = (park_id, act_id,)

    cur.execute(query, params)

In [ ]:
%%time

# calls function to store data for any new park_activites relations
new_parkactivities_df = update_park_activities_db(cur, park_activities_df)

# calls function to insert any new rows into 'park_activities' SQL table
append_parkactivitiesDF_to_table(cur, new_parkactivities_df)

### Insert & Update Park_Topics Data

In [ ]:
# function for inserting and updating data into park_topics_df - to be used with 'park_topics' SQL table
def update_park_topics_db(cur, park_topics_df):
    # empty list for storing new IDs
    new_row = []
    
    for i, row in park_topics_df.iterrows():
        if check_park_topic_existence(cur, row['park_id'], row['top_id']) is None: # if park_topics relation doesn't exist - append to SQL table
            # append details of new row to list for creating dataframe
            new_row.append(row)
        else:
            print('Nothing to insert')
            
    tmp_park_topics_df = pd.DataFrame(new_row)
    return tmp_park_topics_df

In [ ]:
# checks if relationship exists in 'park_topics' SQL table
def check_park_topic_existence(cur, park_id, top_id):
    query = ("""SELECT * FROM park_topics WHERE park_id = (%s) AND topic_id = (%s);""")
    cur.execute(query, (park_id, top_id,))

    return cur.fetchone()

In [ ]:
'''
Function for inserting, row-by-row the new details contained within new_parktopics_df
    - This will call the function 'insert_into_parktopics_table' for actually querying the 'park_topics' SQL table
'''
def append_parktopicsDF_to_table(cur, new_parktopics_df):
    # inserts park_topic relation row-by-row
    for i, row in new_parktopics_df.iterrows():
        insert_into_parktopics_table(cur, row['park_id'], row['top_id'])

In [ ]:
# queries the 'park_topics' SQL table to insert new row(s) of relations gathered
def insert_into_parktopics_table(cur, park_id, top_id):
    query = ("""INSERT INTO park_topics (park_id, topic_id)
                VALUES (%s, %s);""")
    params = (park_id, top_id,)

    cur.execute(query, params)

In [ ]:
%%time

# calls function to store data for any new park_topics relations
new_parktopics_df = update_park_topics_db(cur, park_topics_df)

# calls function to insert any new rows into 'park_topics' SQL table
append_parktopicsDF_to_table(cur, new_parktopics_df)

In [ ]:
# manually close connection after performing needed AWS operations 
conn.close()

# Snowflake

## Pushing Pandas Dataframes to Snowflake Database
- Only INSERT operations performed, there was no intent for records to be updated, so code does not reflect this

In [ ]:
# #local install for Pandas use in Snowflake

# !pip install "snowflake-connector-python[pandas]"

In [ ]:
'''
The following code should be uncommented & used if utilizing a local Jupyter Notebook
  - This is intended to pull stored details for created Snowflake warehouse, for remote local connection 
'''
# # getting Snowflake Auth & Warehouse details
# %run NPS_SNOWFLAKE.ipynb

# user, password, account, warehouse, database, schema = get_nps_snowflake()

In [ ]:
# opening a connection to created NPS database in Snowflake, otherwise raises error
try:
    conn = snowflake.connector.connect(
    user = user,
    password = password,
    account = account,
    warehouse = warehouse,
    database = database,
    schema = schema
    )
except OperationalError as e:
    raise e
else:
    print('Connected!')

In [ ]:
# opening a cursor for performing database operations
cur = conn.cursor()

In [ ]:
'''
Manual setting of Warehouse, Database & Schema to be used
    - Snowflake seems to need this regardless of if initial cursor connection is setup with these params.
'''
cur.execute("USE WAREHOUSE NPS_WH")
cur.execute("USE DATABASE NPS_NATLPARKS")
cur.execute("USE SCHEMA NPS_NATLPARKS.PUBLIC")

### Creating SQL Tables

In [ ]:
# create parks table
create_parks_table = ("""CREATE OR REPLACE TABLE parks(
                        park_id VARCHAR(255) PRIMARY KEY,
                        park_name TEXT NOT NULL,
                        park_state TEXT,
                        park_code VARCHAR(255),
                        park_url TEXT,
                        park_lat VARCHAR(255),
                        park_long VARCHAR(255)
                        )""")

# execute sql command
cur.execute(create_parks_table)

In [ ]:
# create activities table 
create_activities_table = ("""CREATE OR REPLACE TABLE activities (
                    activity_id VARCHAR(255) PRIMARY KEY,
                    activity_name VARCHAR(255) NOT NULL
                )""")

# execute sql command
cur.execute(create_activities_table)

In [ ]:
# create park_activities relationships table
create_park_activities_table = ("""CREATE OR REPLACE TABLE park_activities (
                    park_id VARCHAR(255),
                    activity_id VARCHAR(255),
                    PRIMARY KEY(park_id, activity_id)
                )""")

#execute sql command
cur.execute(create_park_activities_table)

In [ ]:
# create topics table if it doesn't exist already
create_topics_table = ("""CREATE OR REPLACE TABLE topics (
                    topic_id VARCHAR(255) PRIMARY KEY,
                    topic_name VARCHAR(255) NOT NULL
                )""")

# execute sql command
cur.execute(create_topics_table)

In [ ]:
# create park_topics relationships table
create_park_topics_table = ("""CREATE OR REPLACE TABLE park_topics (
                    park_id VARCHAR(255),
                    topic_id VARCHAR(255),
                    PRIMARY KEY(park_id, topic_id)
                )""")

#execute sql command
cur.execute(create_park_topics_table)

### Inserting Data into created Snowflake Tables

In [ ]:
"""
Because Snowflake is case-sensitive and converts high-level nomenclatures (i.e., warehouse, database-name, schema, column-headers)
to upper-case, it is necessary to convert pandas DFs column headers to upper-case for avoiding a ProgramError (invalid identifier)
"""
parks_df.columns = parks_df.columns.str.upper()
activities_df.columns = activities_df.columns.str.upper()
park_activities_df.columns = park_activities_df.columns.str.upper()
topics_df.columns = topics_df.columns.str.upper()
park_topics_df.columns = park_topics_df.columns.str.upper()

In [ ]:
# insert to PARKS table & print success-status of write
park_success, park_nchunks, park_nrows, _ = write_pandas(conn, parks_df, 'PARKS')

park_success

In [ ]:
# renaming columns to match Snowflake target (without impacting AWS code in first sections)
activities_df = activities_df.rename(columns={'ACT_ID': 'ACTIVITY_ID', 'ACT_NAME':'ACTIVITY_NAME'})

In [ ]:
%%time 

# insert to ACTIVITIES table & print success-status of write
activities_success, activities_nchunks, activities_nrows, _ = write_pandas(conn, activities_df, 'ACTIVITIES')

activities_success

In [ ]:
# renaming column to match Snowflake target (without impacting AWS code in first sections)
park_activities_df = park_activities_df.rename(columns={'ACT_ID':'ACTIVITY_ID'})

In [ ]:
%%time 

# insert to PARK_ACTIVITIES table & print success-status of write
park_activities_success, park_activities_nchunks, park_activities_nrows, _ = write_pandas(conn, park_activities_df, 'PARK_ACTIVITIES')

park_activities_success

In [ ]:
# renaming columns to match Snowflake target (without impacting AWS code in first sections)
topics_df = topics_df.rename(columns={'TOP_ID':'TOPIC_ID', 'TOP_NAME':'TOPIC_NAME'})

In [ ]:
%%time 

# insert to TOPICS table & print success-status of write
topics_success, topics_nchunks, topics_nrows, _ = write_pandas(conn, topics_df, 'TOPICS')

topics_success

In [ ]:
# renaming column to match Snowflake target (without impacting AWS code in first sections)
park_topics_df = park_topics_df.rename(columns={'TOP_ID':'TOPIC_ID'})

In [ ]:
%%time 

# insert to PARK_TOPICS table & print success-status of write
park_topics_success, park_topics_nchunks, park_topics_nrows, _ = write_pandas(conn, park_topics_df, 'PARK_TOPICS')

park_topics_success

In [ ]:
# manually close connection after performing needed Snowflake operations 
conn.close()